---

# MessMiner

Agent workflow that mines data out of "messy" sources, and structures it for any use where you need structured data. It is a demo version based on python, for now only extracting data from "ugly" Excel files.

Author: Andre Moreira, November 2025

Stack: 
- Python 3.11; see requirements.txt for dependencies
- LangGraph (orchestration)
- LLMs: Gemini 2.5 Pro, Gemini 2.5 Flash
- pydantic: control structured responses
- Claude Code: co-author

---

## Imports

In [1]:
import json
import os, sys
from dotenv import load_dotenv
import pandas as pd
from io import StringIO
import operator

from pydantic import BaseModel, Field, create_model

from typing import (List, Dict, Any, Optional, TypedDict, Literal, Annotated)

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, RemoveMessage
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool

from langgraph.graph import StateGraph, END
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.constants import Send
from langgraph.graph import add_messages

In [2]:
%load_ext autoreload
%autoreload 2

In [41]:
# Import functions and classes that we use through out this notebook
# Having part of the code elsewhere makes this notebook more readable

from helper_functions import *
from classes import *
from prompts import *

In [45]:
load_dotenv()  # load the content of .env into the environment variables

# Prepare the configuration of the models we will be using
model_config_examiner = {"model":"google_genai:gemini-2.5-pro", "temperature":1, "thinking_budget":-1}
model_config_extractor = {"model":"google_genai:gemini-2.5-pro", "temperature":1, "thinking_budget":-1}

## User inputs

In [5]:
raw_file = 'Example_0.xlsx'

In [6]:
# Optional (but very powerful): provide the agents with some context on the files / information is has to look for
# If you don't want to add any context here, just add as an empty str ""

user_context = "The file contains agricultural data that is almost correctly structured as I want, but it has empty rows; I need this to be a clean, contiguous table"

In [7]:
# Prepare the initial state of the graph

initial_state: OverallState = {"user_background": str(user_context), "file_name": raw_file, "sheets_as_csv": {} }

## Node 1: uploader

Uploads the file, get its structure as csv (text, not as dataframes)

In [8]:
def node_upload_and_organize(state:OverallState) -> OverallState:
    """ Node that solely focuses on uploading a file and organizing its sheets into csv data (in text format)
    """
    
    raw_file = state['file_name']
    state['sheets_as_csv'] = load_excel_as_csv(raw_file)
    
    return state

In [9]:
# For testing, just to see if the node returns the state as we need it
result = node_upload_and_organize(initial_state)

#### Testing

In [31]:
result

{'user_background': 'The file contains agricultural data that is almost correctly structured as I want, but it has empty rows; I need this to be a clean, contiguous table',
 'file_name': 'Example_0.xlsx',
 'sheets_as_csv': {'raw_data_sheet_1': 'Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Cation Exchange Capacity,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,H3A Extraction,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Water Soluble Nitrate,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Pathogenic Fungi,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Beneficial Fungi,Unnamed: 59,Unnamed: 

In [11]:
single_sheet_test=result.get('sheets_as_csv').get('raw_data_sheet_1')

In [12]:
single_sheet_test

'Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Cation Exchange Capacity,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,H3A Extraction,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Water Soluble Nitrate,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Pathogenic Fungi,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Beneficial Fungi,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65\n,Stage,treatment,Sand-Silt-Clay,OM,pH,Nitrate N,Po4P of pH < 6,Po4P of pH > 6,Potassium,Magnesium,Calcium,Sulfur,Zinc,Manganese,Copper,Iron,Boron,Potassium,Magnesium,Calcium,Sodium

## Node 2: examiner

Examines the csv data, extracts possible columns, structures them as pydantic classes for the next node

In [10]:
# Note that the examiner will only receive the part of the OverallState that is actually relevant for it

def examine_in_parallel_node(state: OverallState):
    # Get the user's background from the overall state
    user_background = state['user_background']

    # return the Send pattern with the right bits for the single examiner node
    return [Send("examine_single_csv_node", 
                 {"user_background": user_background, "sheet_name":sheet_name, "sheet_value": sheet_value}) for sheet_name, sheet_value in state["sheets_as_csv"].items()]

def examine_single_csv_node(state: SingleCSVState):
    """ Node that examins a single csv sheet and decides on how to organize it
    """

    # prepare the prompt
    examiner_system_prompt = prepare_examiner_prompt(state['user_background'])
    sheet_value = state['sheet_value']
    
    llm = init_chat_model(**model_config_examiner)
    message_for_llm = [{"role": "system", "content": examiner_system_prompt},
                  {"role": "user", "content": sheet_value}
                  ]

    # the model will return the **structure of the data** as it think it should be extracted
    structured_llm = llm.with_structured_output(SingleCSVStructure)
    response = structured_llm.invoke(message_for_llm)

    response.sheet_name = state['sheet_name']

    update_dict = {"csv_structure": [response]}

    return update_dict

### Testing

While it may be a bit annoying to use jupyter NB real estate to show little response tests, it is a good practice to have this somewhere - the responses are structured in a way that may be a bit confusing, therefore testing the syntax helps later as we run the full graph, nodes, etc, avoiding errors that could be prevented by checking if the responses follow the patterns we want.

#### Single responses

In [13]:
# Testing

examiner_system_prompt = prepare_examiner_prompt(initial_state['user_background'])
csv_data = single_sheet_test

print(examiner_system_prompt)


## Role

You are a senior data scientist with an excellent eye for details.

## Background

The user provided a file containing data structured as comma separated values (csv). 
Besides the csv data, the user also provided this background information: The file contains agricultural data that is almost correctly structured as I want, but it has empty rows; I need this to be a clean, contiguous table

## Task

Your task is to check the csv data passed by the user and identify the data structure that it contains.
the data may or may not be structured as a table, and this is exactly the challenge you need to tackle:
given this csv file, how would you structure the information that it contains in a intelligent table format?
Which columns should you include? Which information should be ignored?

## Instructions

1. Work systematically row by row downwards from the top row
2. Collect the information that will be the basis of the column headers of a structured table




In [14]:
llm = init_chat_model(**model_config_examiner)
message_for_llm = [{"role": "system", "content": examiner_system_prompt},
              {"role": "user", "content": csv_data}]

# the model will return the **structure of the data** as it think it should be extracted
structured_llm = llm.with_structured_output(SingleCSVStructure)

In [15]:
response = structured_llm.invoke(message_for_llm)

In [16]:
response

SingleCSVStructure(fields=[FieldDefinition(name='sample_name', original_name='Unnamed: 0', field_type='str', description='Identifier for the sample, including plot, crop, treatment, and date.', optional=False), FieldDefinition(name='stage', original_name='Stage', field_type='str', description='The growth stage of the plant at the time of sampling.', optional=False), FieldDefinition(name='treatment', original_name='treatment', field_type='str', description='The treatment applied.', optional=False), FieldDefinition(name='sand_silt_clay', original_name='Sand-Silt-Clay', field_type='str', description='Sand-Silt-Clay percentage.', optional=False), FieldDefinition(name='om', original_name='OM', field_type='float', description='Organic Matter percentage.', optional=False), FieldDefinition(name='ph', original_name='pH', field_type='float', description='Soil pH.', optional=False), FieldDefinition(name='nitrate_n', original_name='Nitrate N', field_type='float', description='Nitrate Nitrogen leve

#### Node response (single case)

In [13]:
overall_state_1 = node_upload_and_organize(initial_state)

In [15]:
overall_state_1

{'user_background': 'The file contains agricultural data that is almost correctly structured as I want, but it has empty rows; I need this to be a clean, contiguous table',
 'file_name': 'Example_0.xlsx',
 'sheets_as_csv': {'raw_data_sheet_1': 'Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7\nname,Stage,treatment,Sand-Silt-Clay,OM,pH,Nitrate N,Boron\npl_1 Spinach Control 8/5/19,Pre-product Aplication,Control,51-26-23,2.8,6.8,19.8,1.4\npl_1 Spinach Control 9/10/19,Mid-Growth,Control,46-30-24,2,6.3,63,0.7\npl_1 Spinach Control 9/16/19,Harvest,Control,50-28-22,2.4,6.9,12.6,0.7\npl_1 Spinach Control 10/11/19,Fallow,Control,44-32-24,1.6,7,15.3,1.7\npl_1 Spinach Control 11/7/19,Fallow,Control,48-30-22,1.4,7.5,43.1,1.5\npl_1 Spinach Control 4/27/20,Mid-growth Brassicas,Control,46-30-24,2.6,7.4,39.6,1.4\npl_1 Spianch Lot 11 Control 8/6/20,Harvest,Control,44-28-28,2,6.9,27.9,2\npl_1 Spinach product8/5/19,Pre-product Aplication,product,54-20-26,2.4,7.4,24.

In [16]:
# for testing, build up a initial single csv state that we pass to the node for singular extraction (using Example_0)
single_csv_state_1 = {'user_background':overall_state_1['user_background'], 
                      'sheet_name':'raw_data_sheet_1', 
                      'sheet_value': overall_state_1['sheets_as_csv']['raw_data_sheet_1']}

In [17]:
response_singular = examine_single_csv_node(single_csv_state_1)

In [21]:
response_singular

{'csv_structure': [SingleCSVStructure(sheet_name='raw_data_sheet_1', fields=[FieldDefinition(name='name', original_name='name', field_type='str', description='Identifier for the sample including plot, crop, and date', optional=False), FieldDefinition(name='stage', original_name='Stage', field_type='str', description='The growth stage of the plant', optional=False), FieldDefinition(name='treatment', original_name='treatment', field_type='str', description='The treatment applied', optional=False), FieldDefinition(name='sand', original_name='Sand-Silt-Clay', field_type='int', description='The sand content in the soil', optional=False), FieldDefinition(name='silt', original_name='Sand-Silt-Clay', field_type='int', description='The silt content in the soil', optional=False), FieldDefinition(name='clay', original_name='Sand-Silt-Clay', field_type='int', description='The clay content in the soil', optional=False), FieldDefinition(name='om', original_name='OM', field_type='float', description=

In [19]:
len(response_sigular['csv_structure'])

1

The LLM responds with the right structure (SingleCSVStructure) and the list has only one element, which is exactly how it is supposed to be, because these single element lists will be collected and appended into the corresponding OverallState element (csv_structure as a list of SingleCSVStructure)

## Node 3: extractor

Based on the work done by the node 2, it will now extract the data according to the columns the examiner found as relevant

In [ ]:
# For each CSV, we need to create the pydantic classes
# these will be used to parse the LLM responses, which will then be added to the OverallState as tables

In [47]:

def extract_in_parallel_node(state: OverallState):
    # Get the user's background from the overall state
    user_background = state['user_background']

    # return the Send pattern with the right bits for the single examiner node
    return [Send("extract_single_csv_node", 
                 {"user_background": user_background, 
                  "csv_structure": single_csv_structure,
                  "sheet_name": single_csv_structure.sheet_name,
                  "sheet_value":state['sheets_as_csv'][single_csv_structure.sheet_name]}) for single_csv_structure in state["csv_structure"] ]

def extract_single_csv_node(state: SingleCSVState):
    """ Node that extracts the data from a single csv sheet 
    """

    # Create a class for the table, based on the results from the examiner
    # DynamicModel is now a class that pydantic treats as its BaseModel with the fields
    # that the examiner decided that should be there    
    DynamicModel = build_model_from_structure(state['csv_structure'])
    class TableData(BaseModel):
        rows: list[DynamicModel] 
    
    # prepare the prompt
    extractor_system_prompt = prepare_prompt(extractor_prompt_template, state['user_background'])
    
    sheet_value = state['sheet_value']
    
    llm = init_chat_model(**model_config_extractor)
    message_for_llm = [{"role": "system", "content": extractor_system_prompt},
                  {"role": "user", "content": sheet_value}
                  ]

    # the model will return the **structure of the data** as it think it should be extracted
    structured_llm = llm.with_structured_output(TableData)
    response = structured_llm.invoke(message_for_llm)

    # Finally, the cleaned-up table
    df = pd.DataFrame([row.model_dump() for row in response.rows])

    update_dict = {"cleaned_csv": [df]}

    return update_dict

### Testing

#### Node response (single case)

In [25]:
# still there? if not, run testing in node 2

response_singular['csv_structure'][0]

SingleCSVStructure(sheet_name='raw_data_sheet_1', fields=[FieldDefinition(name='name', original_name='name', field_type='str', description='Identifier for the sample including plot, crop, and date', optional=False), FieldDefinition(name='stage', original_name='Stage', field_type='str', description='The growth stage of the plant', optional=False), FieldDefinition(name='treatment', original_name='treatment', field_type='str', description='The treatment applied', optional=False), FieldDefinition(name='sand', original_name='Sand-Silt-Clay', field_type='int', description='The sand content in the soil', optional=False), FieldDefinition(name='silt', original_name='Sand-Silt-Clay', field_type='int', description='The silt content in the soil', optional=False), FieldDefinition(name='clay', original_name='Sand-Silt-Clay', field_type='int', description='The clay content in the soil', optional=False), FieldDefinition(name='om', original_name='OM', field_type='float', description='The organic matter

In [29]:
# for testing, build up a initial single csv state that we pass to the node for singular extraction (using Example_0)
single_csv_state_2 = {'user_background':overall_state_1['user_background'], 
                      'sheet_name':'raw_data_sheet_1', 
                      'sheet_value': overall_state_1['sheets_as_csv']['raw_data_sheet_1'],
                      'csv_structure':response_singular['csv_structure'][0]
                     }

In [30]:
single_csv_state_2

{'user_background': 'The file contains agricultural data that is almost correctly structured as I want, but it has empty rows; I need this to be a clean, contiguous table',
 'sheet_name': 'raw_data_sheet_1',
 'sheet_value': 'Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7\nname,Stage,treatment,Sand-Silt-Clay,OM,pH,Nitrate N,Boron\npl_1 Spinach Control 8/5/19,Pre-product Aplication,Control,51-26-23,2.8,6.8,19.8,1.4\npl_1 Spinach Control 9/10/19,Mid-Growth,Control,46-30-24,2,6.3,63,0.7\npl_1 Spinach Control 9/16/19,Harvest,Control,50-28-22,2.4,6.9,12.6,0.7\npl_1 Spinach Control 10/11/19,Fallow,Control,44-32-24,1.6,7,15.3,1.7\npl_1 Spinach Control 11/7/19,Fallow,Control,48-30-22,1.4,7.5,43.1,1.5\npl_1 Spinach Control 4/27/20,Mid-growth Brassicas,Control,46-30-24,2.6,7.4,39.6,1.4\npl_1 Spianch Lot 11 Control 8/6/20,Harvest,Control,44-28-28,2,6.9,27.9,2\npl_1 Spinach product8/5/19,Pre-product Aplication,product,54-20-26,2.4,7.4,24.5,1.6\npl_1 Spinach 

In [48]:
response_singular_2 = extract_single_csv_node(single_csv_state_2)

In [51]:
display(response_singular_2['cleaned_csv'][0])

,name,stage,treatment,sand,silt,clay,om,ph,nitrate_n,boron
0,pl_1 Spinach Control 8/5/19,Pre-product Aplication,Control,51,26,23,2.8,6.8,19.8,1.4
1,pl_1 Spinach Control 9/10/19,Mid-Growth,Control,46,30,24,2.0,6.3,63.0,0.7
2,pl_1 Spinach Control 9/16/19,Harvest,Control,50,28,22,2.4,6.9,12.6,0.7
3,pl_1 Spinach Control 10/11/19,Fallow,Control,44,32,24,1.6,7.0,15.3,1.7
4,pl_1 Spinach Control 11/7/19,Fallow,Control,48,30,22,1.4,7.5,43.1,1.5
5,pl_1 Spinach Control 4/27/20,Mid-growth Brassicas,Control,46,30,24,2.6,7.4,39.6,1.4
6,pl_1 Spianch Lot 11 Control 8/6/20,Harvest,Control,44,28,28,2.0,6.9,27.9,2.0
7,pl_1 Spinach product8/5/19,Pre-product Aplication,product,54,20,26,2.4,7.4,24.5,1.6
8,pl_1 Spinach product 9/10/19,Mid-Growth,product,56,24,20,1.7,7.7,51.9,1.0
9,pl_1 Spinach product 9/16/19,Harvest,product,47,28,25,2.4,7.6,29.2,1.1


## Graph

In [ ]:
from IPython.display import Image
from langgraph.graph import END, StateGraph, START

# Construct the graph: here we put everything together to construct our graph
graph = StateGraph(OverallState)
graph.add_node("node_1", node_upload_and_organize)
graph.add_node("node_2", examine_single_csv_node)
graph.add_node("node_3", TBD)

graph.add_edge(START, "node_1")
graph.add_conditional_edges("node_1", examine_in_parallel_node, ["node_2"])
graph.add_edge("node_2", "node_3")
graph.add_edge("node_3", END)

# Compile the graph
IM_workflow = graph.compile()
Image(IM_workflow.get_graph().draw_mermaid_png())